###Descarga e importar librerias


In [ ]:
  !pip install contractions

     |████████████████████████████████| 321 kB 5.1 MB/s 
     |████████████████████████████████| 284 kB 43.4 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85448 sha256=7178038a402cb7e9659d985cb6bece85157952545b6cb3877938d51e34c2ec15
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [ ]:
# Importa la libreria 
import nltk
# Descarga paquetes adicionales

In [66]:
# Importación de librerias
seed = 161
import pandas as pd
import numpy as np
import sys
from pandas_profiling import ProfileReport

import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report, confusion_matrix, plot_precision_recall_curve
from sklearn.base import BaseEstimator, ClassifierMixin
# Para crear el arbol de decisión 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neighbors import KNeighborsClassifier
# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score 
from sklearn.metrics import plot_confusion_matrix

# Regresion lineal
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
# Preprocesar
from sklearn.preprocessing import StandardScaler, Normalizer
# Balanceo
from imblearn.over_sampling import SMOTE

# Para evaluar el modelo
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix
# Seaborn
import seaborn as sns 
from sklearn import tree
from pylab import rcParams
import matplotlib.pyplot as plt

In [ ]:
# Punkt permite separar un texto en frases.
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Descarga todas las palabras vacias, es decir, aquellas que no aportan nada al significado del texto
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Descarga de paquete WordNetLemmatizer, este es usado para encontrar el lema de cada palabra
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

###Lectura y perfilamiento de datos


In [ ]:
data_train=pd.read_csv('/content/train.txt', sep=';', encoding = 'utf-8',header=None)
# Dimensiones de los datos
data_train.shape

(16000, 2)

In [ ]:
data_test=pd.read_csv('/content/test.txt', sep=';', encoding = 'utf-8',header=None)
# Dimensiones de los datos
data_test.shape

(2000, 2)

In [ ]:
data_val=pd.read_csv('/content/val.txt', sep=';', encoding = 'utf-8',header=None)
# Dimensiones de los datos
data_val.shape

(2000, 2)

In [ ]:
data_train[1]=data_train[1].replace({'sadness': 1, 'anger': 2, 'love': 3, 'joy': 4, 'fear': 5, 'surprise':6})
data_test[1]=data_test[1].replace({'sadness': 1, 'anger': 2, 'love': 3, 'joy': 4, 'fear': 5, 'surprise':6})
data_val[1]=data_val[1].replace({'sadness': 1, 'anger': 2, 'love': 3, 'joy': 4, 'fear': 5, 'surprise':6})

In [ ]:
textos = data_train.copy()
textos['Conteo'] = [len(x) for x in textos[0]]
textos['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in textos[0]]
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos[0]]
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos[0]]

In [ ]:
textos = data_test.copy()
textos['Conteo'] = [len(x) for x in textos[0]]
textos['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in textos[0]]
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos[0]]
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos[0]]

In [ ]:
textos = data_val.copy()
textos['Conteo'] = [len(x) for x in textos[0]]
textos['Moda'] = [max(set(x.split(' ')), key = x.split(' ').count) for x in textos[0]]
textos['Max'] = [[max([len(x) for x in i.split(' ')])][0] for i in textos[0]]
textos['Min'] = [[min([len(x) for x in i.split(' ')])][0] for i in textos[0]]

In [ ]:
# Se realiza un perfilamiento de los datos con la libre pandas profiling
#ProfileReport(textos)

# Nota: Este comando puede generar el siguiente error: 
# ImportError: cannot import name 'ABCIndexClass' from 'pandas.core.dtypes.generic'
# Para solucionarlo, basta instalar cualquie version 1.2.x de Pandas: 

###Preparacion de datos


limpieza


In [ ]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words
    

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words


def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    return words

tokenizacion

In [ ]:
#Aplica la corrección de las contracciones
data_train[0] = data_train[0].apply(contractions.fix) 
data_test[0] = data_test[0].apply(contractions.fix) 
data_val[0] = data_val[0].apply(contractions.fix) 
 #Aplica la eliminación del ruido
data_train['words'] = data_train[0].apply(word_tokenize).apply(preprocessing)
data_test['words'] = data_test[0].apply(word_tokenize).apply(preprocessing)
data_val['words'] = data_val[0].apply(word_tokenize).apply(preprocessing)

normalizacion

In [ ]:
def stem_words(words):
    """Stem words in list of tokenized words"""
#    stemmer = LancasterStemmer()
#    stems = []
#    for word in words:
#        stem = stemmer.stem(word)
#        stems.append(stem)
#    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def stem_and_lemmatize(words):
    lemmas = lemmatize_verbs(words)
    return lemmas

#Aplica lematización y Eliminación de Prefijos y Sufijos.
data_train['words'] = data_train['words'].apply(stem_and_lemmatize) 
data_test['words'] = data_test['words'].apply(stem_and_lemmatize) 
data_val['words'] = data_val['words'].apply(stem_and_lemmatize) 

In [ ]:
data_train['words']

0                                        [feel, humiliate]
1        [go, feel, hopeless, damn, hopeful, around, so...
2                [grab, minute, post, feel, greedy, wrong]
3        [ever, feel, nostalgic, fireplace, know, still...
4                                          [feel, grouchy]
                               ...                        
15995    [brief, time, beanbag, say, anna, feel, like, ...
15996    [turn, feel, pathetic, still, wait, table, sub...
15997                        [feel, strong, good, overall]
15998                    [feel, like, rude, comment, glad]
15999                   [know, lot, feel, stupid, portray]
Name: words, Length: 16000, dtype: object

seleccion de campos

In [ ]:
data_train['processed_message'] = data_train['words'].apply(lambda x: ' '.join(map(str, x)))
X_data= data_train['processed_message']
y_data = data_train[1]

matriz de conteo

In [ ]:
count = CountVectorizer()
X_count = count.fit_transform(X_data)
print(X_count.shape)
X_count.toarray()[0]

(16000, 12061)


array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
data_test['processed_message'] = data_test['words'].apply(lambda x: ' '.join(map(str, x)))
X_test = count.fit_transform(data_test['processed_message'])
print(X_test.shape)
X_test.toarray()[0]

(2000, 3808)


array([0, 0, 0, ..., 0, 0, 0])

Matriz TF-ID

In [ ]:
 vectorizer = TfidfVectorizer()
 X_ID = vectorizer.fit_transform(X_data)

In [ ]:
print(X_ID)

  (0, 5090)	0.9889069406128232
  (0, 3887)	0.14853640229851484
  (1, 767)	0.44540584507336783
  (1, 1524)	0.32075393333556007
  (1, 9854)	0.3043171823000295
  (1, 580)	0.29147433552867913
  (1, 5014)	0.3888011381197094
  (1, 2517)	0.4127373380836858
  (1, 5016)	0.3840759022899946
  (1, 4477)	0.22034234972378397
  (1, 3887)	0.05984000882412842
  (2, 11937)	0.3905048348651996
  (2, 4574)	0.43381526581073987
  (2, 8178)	0.362690236072352
  (2, 6772)	0.4978583161827672
  (2, 4522)	0.5247650992815985
  (2, 3887)	0.0672664057154502
  (3, 8387)	0.5458231231359385
  (3, 10123)	0.25906570528480016
  (3, 5922)	0.22822561411666759
  (3, 3987)	0.5592502350171039
  (3, 7302)	0.40491074463378485
  (3, 3609)	0.320064563124414
  (3, 3887)	0.06099415219424682
  (4, 4603)	0.9907100239386764
  :	:
  (15995, 10756)	0.20596454482266746
  (15995, 3887)	0.054234349363171765
  (15996, 10233)	0.47460177710630275
  (15996, 2666)	0.40197172855899893
  (15996, 10456)	0.38317499332382915
  (15996, 10531)	0.3573543

Asiganacion de ejes


In [ ]:
# X la matriz y Y el emoji
X = X_ID
Y = data_train[1]

In [ ]:
# Parte en Entreno y Prueba
#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
X_train=X
X_test= X_test
Y_train = Y
Y_test = data_test[1]

###KNN


In [ ]:
# Modelo
knn = KNeighborsClassifier(n_neighbors=9)

In [ ]:
# Entrenar el modelo
knn.fit(X_train,Y_train)

# Predicciones
y_pred_train = knn.predict(X_train)
y_pred_test = knn.predict(X_test)



In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
# Lista de Hiperparámetros a afinar
n_neighbors = list(range(1,10))
p=[1,2] #Función de distancia 1: manhattan, 2: euclidean, otro valor: minkowski

#Convert to dictionary
hyperparameters = dict(n_neighbors=n_neighbors, p=p)

#Create new KNN object
knn_2 = KNeighborsClassifier()

#Use GridSearch
mejor_modelo_knn = GridSearchCV(knn_2, hyperparameters, cv=10)

#Fit the model
mejor_modelo_knn.fit(X_train, Y_train)

#Print The value of best Hyperparameters
print('Best p:', mejor_modelo_knn.best_estimator_.get_params()['p'])
print('Best n_neighbors:', mejor_modelo_knn.best_estimator_.get_params()['n_neighbors'])

Best p: 2
Best n_neighbors: 9


In [ ]:
print(classification_report(Y_train, y_pred_train))

              precision    recall  f1-score   support

           1       0.82      0.94      0.88      4666
           2       0.86      0.83      0.85      2159
           3       0.84      0.66      0.74      1304
           4       0.87      0.90      0.89      5362
           5       0.91      0.75      0.82      1937
           6       0.82      0.61      0.70       572

    accuracy                           0.85     16000
   macro avg       0.85      0.78      0.81     16000
weighted avg       0.86      0.85      0.85     16000



###Arboles


In [58]:
# Primero declaramos el preprocesamiento
pre_process = [('normalizer', Normalizer()),
               ('standarizer', StandardScaler())] 

In [59]:
# Classificador
classfier = [('tree',DecisionTreeClassifier(random_state=seed))]

In [63]:
# Modelo
arbol = DecisionTreeClassifier(criterion='entropy', max_depth = 4, random_state=0)

# Entrenar el modelo
arbol = arbol.fit(X_train,Y_train)

# Predicciones
y_pred_train = arbol.predict(X_train)
y_pred_test = arbol.predict(X_test)


In [ ]:
print(classification_report(Y_train, y_pred_train))

              precision    recall  f1-score   support

           1       0.50      0.00      0.00      4666
           2       1.00      0.00      0.00      2159
           3       0.75      0.00      0.00      1304
           4       0.34      0.99      0.51      5362
           5       0.51      0.07      0.12      1937
           6       0.68      0.29      0.40       572

    accuracy                           0.35     16000
   macro avg       0.63      0.22      0.17     16000
weighted avg       0.54      0.35      0.20     16000



In [73]:
test = []
for i in range(10):
    # Split
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=i)
    
    # Declare
    arbol.fit(X_train, Y_train)
    y_pred_test = arbol.predict(X_test)
    
print('F1 Score sobre test: %.3f' % np.mean(test))

F1 Score sobre test: nan


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [76]:
plot_confusion_matrix(arbol, X_test, Y_test)  
plt.show()

##SVC

In [ ]:
>>> from sklearn.svm import LinearSVC
>>> from sklearn.pipeline import make_pipeline
>>> from sklearn.preprocessing import StandardScaler
>>> from sklearn.datasets import make_classification
X_train, Y_train = make_classification(n_features=4, random_state=0)
clf = make_pipeline(StandardScaler(),
                     LinearSVC(random_state=0, tol=1e-5))
clf.fit(X_train, Y_train)
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

In [ ]:
y_pred_train = clf.predict(X_train)

In [ ]:
# Metricas
print(classification_report(Y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93        50
           1       0.96      0.90      0.93        50

    accuracy                           0.93       100
   macro avg       0.93      0.93      0.93       100
weighted avg       0.93      0.93      0.93       100

